In [1]:
# Import glob, a module that helps with file management.
import glob
import pandas as pd
from spellchecker import SpellChecker
from pathlib import Path
from nltk import wordpunct_tokenize
from collections import Counter
import os
import re
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import numpy as np
import ipynbname

In [2]:
# Define the output path and create the directory if it doesn't exist
outputpath = "./final"
outputfile_path = Path(outputpath)
outputfile_path.mkdir(exist_ok=True)

# Get the current working directory
texts_folder = Path.cwd()

# Find all .txt files in the current directory
texts_list = sorted(glob.glob("*.txt"))
print("Text files to be spellchecked:", texts_list)

current_file = ipynbname.name().split(' ')[0]

current_folder = os.path.basename(os.getcwd())

print(f"current_folder = '{current_folder}'")

Text files to be spellchecked: ['République.txt']
current_folder = 'Concatenated'


In [3]:
# Load the spellchecker dictionary.
# Replace the language attribute with another 2 letter code
# to select another language. Options are: English - ‘en’, Spanish - ‘es’,
# French - ‘fr’, Portuguese - ‘pt’, German - ‘de’, Russian - ‘ru’.

spell = SpellChecker(language='fr')

In [4]:
# Initialize an empty list to collect data
data_list = []

with pd.ExcelWriter(f'{current_file}_spellcheck_data.xlsx', engine='openpyxl') as writer:
    for txt_file in sorted(texts_folder.glob('*.txt')):
        
        # Extract the relevant parts of the file path
        parts = txt_file.parts[-3:]
        file_name = os.path.join(parts[0], parts[1], parts[2])
        
        # Open each text file and read text into `ocrText`
        with open(txt_file, 'r') as inputFile:
            ocrText = inputFile.read()
            
        # Join hyphenated words that are split between lines
        ocrText = ocrText.replace("-\n", "")
        
        # Tokenize the text
        tokens = wordpunct_tokenize(ocrText)
        
        # Lowercase all tokens and filter out non-alphabetic tokens
        tokens = [token.lower() for token in tokens if token.isalpha()]
        
        # Identify unknown words
        spell = SpellChecker()
        unknown = spell.unknown(tokens)
        
        # Calculate readability
        if len(unknown) != 0:
            readability = round(100 - (float(len(unknown)) / float(len(tokens)) * 100), 2)
        else:
            readability = 100
        
        # Sort the unknown words first by 'counts' descending and then by 'unknown_tokens' alphabetically
        sorted_unknown = sorted(Counter(unknown).items(), key=lambda item: (-item[1], item[0]))
        
        # Append the data for this file to the list
        data_list.append({
            "file_name": file_name,
            "token_count": len(tokens),
            "unknown_count": len(unknown),
            "readability": readability,
        })
        
        # Write the DataFrame to a new sheet in the Excel file
        pd.DataFrame([data_list[-1]]).to_excel(writer, sheet_name=os.path.splitext(txt_file.name)[0], index=False)
        
        # Write the unknown tokens and counts vertically
        sheet = writer.sheets[os.path.splitext(txt_file.name)[0]]
        start_row = 6  # Add two extra rows of space
        sheet.cell(row=start_row, column=1, value="unknown_tokens").font = Font(bold=True)
        sheet.cell(row=start_row, column=2, value="counts").font = Font(bold=True)
        for i, (token, count) in enumerate(sorted_unknown, start=start_row + 1):
            sheet.cell(row=i, column=1, value=token)
            sheet.cell(row=i, column=2, value=count)
        
        # Print a message indicating the file has been processed
        print(txt_file, "checked for readability.")
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data_list)

  
    # Write the combined DataFrame to a summary sheet
    df.to_excel(writer, sheet_name='Summary', index=False)

# Load the workbook to adjust cell alignment
wb = load_workbook(f'{current_file}_spellcheck_data.xlsx')

# Align cells in the summary sheet
summary_sheet = wb['Summary']
for row in summary_sheet.iter_rows(min_row=2, max_row=summary_sheet.max_row, min_col=1, max_col=4):
    for cell in row:
        cell.alignment = Alignment(vertical='top')

# Align cells in each individual sheet
for sheet_name in wb.sheetnames:
    if sheet_name != 'Summary':
        sheet = wb[sheet_name]
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=4):
            for cell in row:
                cell.alignment = Alignment(vertical='top')

# Save the workbook with the updated alignment
wb.save(f'{current_file}_spellcheck_data.xlsx')

df.head(20)

/home/lucas-jerusalimiec/Documents/OCR Text/Text/Bodin/République/Concatenated/République.txt checked for readability.


,file_name,token_count,unknown_count,readability
0,République/Concatenated/République.txt,354867,43556,87.73


In [5]:
# Sorted unknown_words
unknown_words = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'aduis', 'ainfi', 'ariftocratique', 'bourgeoifie', 'angle- \nterre',
    'celte', 'cha- \ncun', 'charle ', 'ciuile', 'ciuiles',
    'ci- \ntoyens', 'cinqviesme', 'confeil',
    'deflus', 'deffus', 'deliberatiue', 'deuant', 'doibt', 'don- \nner',
    'efcus', 'effre', 'eflat', 'eftat', 'eftats', 'em pereur', 'em- \npereur', 'em- \npire', 'empe- \nreur',
    'enco- \nres', ' és ', 'euure',
    ' fang ', 'feigneuriale', 'fenat', ' fept ', 'feulement', 'fleuriffant', 'fouuent',
    'fouuerain', 'fouucrain', 'fouueraineté', 'fouverain',
    'fou- \nuerain', 'fouue- \nrain',
    'fouuerai- \nneté', 'fuget', 'fugets', 'fu- \ngets', 'fuft', 'gouuernement',
    'iuflice', 'iufte', 'iuftice', 'iuf- \nques', ' liue ', 'liures', 'loüys', 'macciauel', 'mai- \nftre',
    'magi- \nftrats', 'magiftrat', 'magiftrats', 'maniment', 'monftré', 'mo- \nnarchie',
    'nousauons', 'paffer', 'parti- \nculier', 'particu- \nlier', 'pauureté', 'peu- \nple', 'per- \nfonne', 'plu- \nfieurs',
    'plu- \nficurs', 'plufieurs', 'pre- \nmier', 'priué', 'pu- \nblic',
    'puif- \nfance', 'puif- \nlance', 'puiffance', 'puiflance',
    'queftion', ' quia ', 'qvatriesme', 'republi- \nques', 'repvbliqve',
    'repu- \nblique', 'septétrion', ' toft ', 'tou- \ntesfois', 'touf- \niours', 'toufiours', 'vns'
]

# Sorted known_words
known_words = [
    '', '', '', '', '', '', '', '', '', '',
    'advis', 'ainsi', 'aristocratique', 'bourgeoisie', 'angleterre',
    'ceste', 'chacun', 'charles ', 'civile', 'civiles',
    'citoyens', 'cinquiesme', 'conseil',
    'dessus', 'dessus', 'deliberative', 'deuant', 'doit', 'donner',
    'escus', 'estre', 'estat', 'estat', 'estats', 'empereur', 'empereur', 'empire', 'empereur',
    'encores', ' les ', 'oeuvre',
    ' sang ', 'seigneuriale', 'senat', ' sept ', 'seulement', 'fleurissant', 'souvent',
    'souverain', 'souverian', 'souveraineté', 'souverain',
    'souverain', 'souverain',
    'souveraineté', 'sujet', 'sujets', 'sujets', 'fust', 'gouvernement',
    'iustice', 'iuste', 'iustice', 'iusques', 'live', 'livres', 'louis', 'macciavel', 'maistre',
    'magistrats', 'magistrat', 'magistrats', 'maniement', 'monstré', 'monarchie',
    'nous avons', 'passer', 'particulier', 'particulier', 'pauvreté', 'peuple', 'personne', 'plusieurs',
    'plusieurs', 'plusieurs', 'premier', 'privé', 'public',
    'puissance', 'puissance', 'puissance', 'puissance',
    'question', ' qui a ', 'quatriesme', 'republiques', 'republique',
    'republique', 'septentrion', 'tost', 'toutesfois', 'tousjours', 'tousjours', 'uns'
]


In [6]:
first_file_iteration = True  # Flag to control the print statement

for file in texts_list:
    
    # Identify the output file path for each text file
    outputfile = f'{outputfile_path}/{Path(file).stem}_corrected.txt'
    
    # Open a file in "read" (r) mode
    with open(file, "r") as text:
        # Read in the contents of that file
        word_correction = text.read()

    word_correction = word_correction.lower()
    
    # Find instances of an unknown word and replace with a known word
    for i in range(len(known_words)):
        unknown_word = unknown_words[i]
        known_word = known_words[i]
        word_correction = word_correction.replace(unknown_word, known_word)
        
        if first_file_iteration:
            print(f"All instances of '{unknown_word}' replaced with '{known_word}'.")
    
    # Open the file in "write" (w) mode
    with open(outputfile, "w") as file:
        # Add the changed word into the reopened file
        file.write(word_correction)
    
    print(f"Corrected file saved as {outputfile}.")
    first_file_iteration = False  # Disable the flag after the first file

All instances of '0' replaced with ''.
All instances of '1' replaced with ''.
All instances of '2' replaced with ''.
All instances of '3' replaced with ''.
All instances of '4' replaced with ''.
All instances of '5' replaced with ''.
All instances of '6' replaced with ''.
All instances of '7' replaced with ''.
All instances of '8' replaced with ''.
All instances of '9' replaced with ''.
All instances of 'aduis' replaced with 'advis'.
All instances of 'ainfi' replaced with 'ainsi'.
All instances of 'ariftocratique' replaced with 'aristocratique'.
All instances of 'bourgeoifie' replaced with 'bourgeoisie'.
All instances of 'angle- 
terre' replaced with 'angleterre'.
All instances of 'celte' replaced with 'ceste'.
All instances of 'cha- 
cun' replaced with 'chacun'.
All instances of 'charle ' replaced with 'charles '.
All instances of 'ciuile' replaced with 'civile'.
All instances of 'ciuiles' replaced with 'civiles'.
All instances of 'ci- 
toyens' replaced with 'citoyens'.
All instances 

In [7]:
import locale

# Set the locale to your desired setting (e.g., 'fr_FR.UTF-8' for French)
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

# Assume spell is a previously defined spell checker instance
# outputfile_path is the directory path to save the Excel file
# outputpath is the directory path containing the .txt files

# Create an Excel writer object
with pd.ExcelWriter(f'{outputfile_path}/{current_file}_corrected_spellcheck_data.xlsx', engine='openpyxl') as writer:

    # Get list of files and sort them alphabetically using locale-aware sorting
    files = [f for f in os.listdir(outputpath) if f.endswith('.txt')]
    sorted_files = sorted(files, key=locale.strxfrm)

    for filename in sorted_files:
        file_path = os.path.join(outputpath, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text_data = file.read()

        text_data = text_data.replace("-\n", "")

        words = wordpunct_tokenize(text_data)

        misspelled = spell.unknown(words)

        if len(misspelled) != 0:
            readability = round(100 - (float(len(misspelled)) / float(len(words)) * 100), 2)
        else:
            readability = 100

        # Count the frequency of each misspelled word
        word_counts = Counter(misspelled)

        # Sort word_counts first by 'count' descending and then by 'word' alphabetically
        sorted_word_counts = sorted(word_counts.items(), key=lambda item: (-item[1], item[0]))

        # Create a DataFrame and sort by frequency
        misspelled_df = pd.DataFrame(sorted_word_counts, columns=['word', 'count'])

        # Add readability as the first row
        readability_header = pd.DataFrame([["Readability", readability]], columns=['word', 'count'])
        blank_rows = pd.DataFrame([["", ""]], columns=['word', 'count'])
        header_row = pd.DataFrame([["word", "count"]], columns=['word', 'count'])
        misspelled_df = pd.concat([readability_header, blank_rows, header_row, misspelled_df], ignore_index=True)

        # Write the DataFrame to a new sheet in the Excel file
        sheet_name = os.path.splitext(filename)[0]
        misspelled_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

        print(f'Spellchecked {filename}. Readability = {readability}')

print(f"All data combined into {outputfile_path}/{current_file}_corrected_spellcheck_data.xlsx")


Spellchecked République_corrected.txt. Readability = 90.55
All data combined into final/République_corrected_spellcheck_data.xlsx
